In [23]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [24]:
#web-traffic
def create_ts_data(data, window_size, target_size):
    i = 1
    while i < window_size:
        data["users_{}".format(i)] = data["users"].shift(-i)
        i += 1
    i = 0
    while i < target_size:
        data["target_{}".format(i)] = data["users"].shift(-i-window_size)
        i += 1
    data = data.dropna(axis=0)
    return data

def create_enhanced_features(data):
    data['dayofweek'] = data['date'].dt.dayofweek
    data['month'] = data['date'].dt.month
    data['is_weekend'] = data['dayofweek'].isin([5,6]).astype(int)
    return data

data = pd.read_csv("week9/web-traffic.csv")
data["date"] = pd.to_datetime(data["date"], format="%d/%m/%y", dayfirst=True)

data = create_enhanced_features(data)

# print(data.info())

# # Plot Figure
# fig, ax = plt.subplots()
# ax.plot(data["date"], data["users"])
# ax.set_xlabel("Date")
# ax.set_ylabel("Traffic")
# fig.show()

window_size = 5
target_size = 3
train_ratio = 0.8
data = create_ts_data(data, window_size, target_size)
targets = ["target_{}".format(i) for i in range(target_size)]
x = data.drop(["date"] + targets, axis=1)
y = data[targets]
num_samples = len(data)
x_train = x[:int(num_samples*train_ratio)]
y_train = y[:int(num_samples*train_ratio)]
x_test = x[int(num_samples*train_ratio):]
y_test = y[int(num_samples*train_ratio):]

models = [RandomForestRegressor() for _ in range(target_size)]
for index, model in enumerate(models):
    model.fit(x_train, y_train["target_{}".format(index)])


maes = []
mses = []
r2s = []
for index, model in enumerate(models):
    y_predict = model.predict(x_test)
    maes.append(mean_absolute_error(y_test["target_{}".format(index)], y_predict))
    mses.append(mean_squared_error(y_test["target_{}".format(index)], y_predict))
    r2s.append(r2_score(y_test["target_{}".format(index)], y_predict))

print("MAEs: ",maes)
print("MSEs: ",mses)
print("R2s: ",r2s)

# Test
# Sửa lại phần dự đoán để đảm bảo cùng thứ tự
current_data = [2525, 2479, 2246, 2345, 2555]
test_date = '15/07/20'
test_date = pd.to_datetime(test_date, format='%d/%m/%y')

# Tạo DataFrame với tất cả features
base_features = ['users'] + [f'users_{i}' for i in range(1, window_size)]
pred_data = pd.DataFrame([current_data], columns=base_features)
pred_data['dayofweek'] = test_date.dayofweek
pred_data['month'] = test_date.month
pred_data['is_weekend'] = int(test_date.dayofweek in [5,6])

# Sắp xếp lại các cột theo đúng thứ tự của training data
pred_data = pred_data[x_train.columns]
print(pred_data)

# Dự đoán
predictions = []
for index, model in enumerate(models):
    pred = model.predict(pred_data)
    predictions.append(pred[0])
    print(f"Dự đoán số users cho ngày thứ {index + 1} kế tiếp: {pred[0]:.0f}")

MAEs:  [179.9401204819277, 204.83289156626512, 243.76530120481934]
MSEs:  [60348.923268674705, 83389.52980843371, 100275.97483614457]
R2s:  [0.9063323908688211, 0.8663959945467108, 0.8326300954597936]
   users  dayofweek  month  is_weekend  users_1  users_2  users_3  users_4
0   2525          2      7           0     2479     2246     2345     2555
Dự đoán số users cho ngày thứ 1 kế tiếp: 2788
Dự đoán số users cho ngày thứ 2 kế tiếp: 2768
Dự đoán số users cho ngày thứ 3 kế tiếp: 2674


In [27]:
#bike-sharing-dataset
def create_ts_data_bike(data, window_size, target_size):
    i = 1
    while i < window_size:
        data[f"users_{i}"] = data["users"].shift(-i)
        i += 1
    i = 0
    while i < target_size:
        data[f"target_{i}"] = data["users"].shift(-i-window_size)
        i += 1
    data = data.dropna(axis=0)
    return data

bike_data = pd.read_csv("week9/bike-sharing-dataset.csv")
bike_data["date_time"] = pd.to_datetime(bike_data["date_time"])
# Xử lý dữ liệu category
# Chuyển đổi weather thành dạng số
weather_mapping = {
    'clear': 1,
    'mist': 2,
    'rain': 3
}
bike_data['weather'] = bike_data['weather'].map(weather_mapping)

# print(bike_data.info())

bike_window_size = 24  # 24 giờ trước đó
bike_target_size = 12  # Dự đoán 12 giờ tiếp theo
bike_train_ratio = 0.8

features_to_use = ['users', 'holiday', 'workingday', 'weather', 
                   'temp', 'atemp', 'hum', 'windspeed', 
                   'hour', 'weekday', 'month']

bike_data_model = bike_data[features_to_use].copy()
bike_data_model = create_ts_data_bike(bike_data_model, bike_window_size, bike_target_size)

# Tách features và targets
bike_targets = [f"target_{i}" for i in range(bike_target_size)]
bike_feature_cols = [col for col in bike_data_model.columns if col not in bike_targets]
x_bike = bike_data_model[bike_feature_cols]
y_bike = bike_data_model[bike_targets]

# Chia tập train/test
bike_num_samples = len(bike_data_model)
x_train_bike = x_bike[:int(bike_num_samples*bike_train_ratio)]
y_train_bike = y_bike[:int(bike_num_samples*bike_train_ratio)]
x_test_bike = x_bike[int(bike_num_samples*bike_train_ratio):]
y_test_bike = y_bike[int(bike_num_samples*bike_train_ratio):]

# Huấn luyện mô hình
bike_models = [RandomForestRegressor(random_state=100)  for _ in range(bike_target_size)]

for index, model in enumerate(bike_models):
    model.fit(x_train_bike, y_train_bike[f"target_{index}"])

# Đánh giá mô hình
bike_maes = []
bike_mses = []
bike_r2s = []
for index, model in enumerate(bike_models):
    y_pred = model.predict(x_test_bike)
    bike_maes.append(mean_absolute_error(y_test_bike[f"target_{index}"], y_pred))
    bike_mses.append(mean_squared_error(y_test_bike[f"target_{index}"], y_pred))
    bike_r2s.append(r2_score(y_test_bike[f"target_{index}"], y_pred))

print("MAEs: ", [round(mae, 2) for mae in bike_maes])
print("MSEs: ", [round(mse, 2) for mse in bike_mses])
print("R2s: ", [round(r2, 4) for r2 in bike_r2s])

MAEs:  [31.5, 41.43, 46.87, 50.42, 52.83, 53.36, 54.07, 54.17, 55.14, 55.78, 56.45, 57.74]
MSEs:  [2665.2, 4406.78, 5647.75, 6477.08, 6996.23, 7141.86, 7417.94, 7321.06, 7523.69, 7856.46, 8235.39, 8586.2]
R2s:  [0.9454, 0.9097, 0.8842, 0.8669, 0.856, 0.8529, 0.8472, 0.8492, 0.845, 0.8382, 0.8304, 0.8231]
